In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_validate, KFold, LeaveOneOut
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler, PowerTransformer

from scipy.stats.mstats import winsorize

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

from datetime import datetime

import seaborn as sns

# Data processing

## Get data

In [2]:
df_raw = pd.read_csv('exploratory.csv')

print(df_raw.dtypes)

ID_PATH_FLOWMONTH           int64
ISO                        object
PEAKTYPE                   object
HEDGETYPE                  object
SOURCEID                    int64
SINKID                      int64
CLUSTERID_LOW_SRC          object
CLUSTERID_LOW_SNK          object
SOURCE_ZONE                object
SINK_ZONE                  object
MW_NET                    float64
MW_BUY                    float64
MW_SELL                   float64
DC_MCP_WAVG               float64
DC_AUCTION_TYPE_FIRST      object
DC_AUCTION_MONTH_FIRST     object
FW_DC_FIRST                 int64
FLOWMONTH                  object
AUCTION_ID                 object
AUCTION_MONTH              object
AUCTION_TYPE               object
FW                          int64
MCP                       float64
CONG                      float64
FUTURE_AUCTION_ID          object
FUTURE_AUCTION_MONTH       object
FUTURE_AUCTION_TYPE        object
FUTURE_FW                 float64
FUTURE_MCP                float64
dtype: object


In [3]:
df_raw.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,AUCTION_MONTH,AUCTION_TYPE,FW,MCP,CONG,FUTURE_AUCTION_ID,FUTURE_AUCTION_MONTH,FUTURE_AUCTION_TYPE,FUTURE_FW,FUTURE_MCP
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,2023-02-01,Mar,1,-0.07,0.2016,PJMISO:Apr:1:2023-03-01,2023-03-01,Apr,0.0,1.05
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,2023-05-01,Jun,2,23.79,46.4896,PJMISO:Aug:1:2023-07-01,2023-07-01,Aug,0.0,19.41
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,2023-05-01,Jun,2,23.79,46.4896,PJMISO:Jul:1:2023-06-01,2023-06-01,Jul,1.0,20.41
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,2023-06-01,Jul,1,20.41,46.4896,PJMISO:Aug:1:2023-07-01,2023-07-01,Aug,0.0,19.41
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,2023-08-01,Sep,3,-5.88,-0.4024,PJMISO:Dec:1:2023-11-01,2023-11-01,Dec,0.0,-4.56


In [4]:
df_raw.describe()

,ID_PATH_FLOWMONTH,SOURCEID,SINKID,MW_NET,MW_BUY,MW_SELL,DC_MCP_WAVG,FW_DC_FIRST,FW,MCP,CONG,FUTURE_FW,FUTURE_MCP
count,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.019640e+06,2.019640e+06
mean,3.260349e+16,4.384865e+08,4.472235e+08,1.054111e+01,1.428585e+01,3.744738e+00,2.616037e+01,8.869535e+00,5.589818e+00,3.299981e+01,1.619917e+01,2.311615e+00,2.940816e+01
std,5.322809e+18,6.926265e+08,6.915405e+08,1.837164e+01,1.640692e+01,8.016405e+00,3.213635e+02,2.451281e+00,2.603479e+00,3.396788e+02,5.640391e+02,2.196493e+00,3.232869e+02
min,-9.223338e+18,4.882400e+04,4.882400e+04,-1.828000e+02,0.000000e+00,0.000000e+00,-5.760430e+03,2.000000e+00,1.000000e+00,-6.740720e+03,-2.704318e+04,0.000000e+00,-1.097964e+04
25%,-4.577779e+18,2.160179e+07,3.106543e+07,0.000000e+00,4.200000e+00,0.000000e+00,-2.519202e+01,7.000000e+00,4.000000e+00,-2.441000e+01,-3.001840e+01,0.000000e+00,-2.505000e+01
50%,8.058980e+16,3.836801e+07,4.446086e+07,8.000000e+00,9.100000e+00,0.000000e+00,-3.110000e+00,9.000000e+00,6.000000e+00,1.720000e+00,2.592000e-01,2.000000e+00,2.110000e+00
75%,4.634187e+18,1.084391e+09,1.084391e+09,1.650000e+01,1.880000e+01,5.100000e+00,1.611641e+01,1.100000e+01,8.000000e+00,3.515000e+01,3.757760e+01,4.000000e+00,3.731000e+01
max,9.223273e+18,2.156114e+09,2.156114e+09,4.718000e+02,4.718000e+02,1.828000e+02,1.337885e+04,1.200000e+01,1.100000e+01,1.369067e+04,2.636686e+04,1.000000e+01,1.780315e+04


## Placeholder for subsetting

In [5]:
df_sample = df_raw[
    (df_raw['MW_NET'] != 0) & ## Drop paths that they bought up and totally exited (like an annual flip)
    (df_raw['DC_MCP_WAVG'] != 0) &
    (df_raw['HEDGETYPE'] == 'Obligation')
].copy()

df_sample = df_sample.filter(items=[
    'ID_PATH_FLOWMONTH',
    'ISO',
    'PEAKTYPE',
    'HEDGETYPE',
    'SOURCEID',
    'SINKID',
    'CLUSTERID_LOW_SRC',
    'CLUSTERID_LOW_SNK',
    'SOURCE_ZONE',
    'SINK_ZONE',
    'MW_NET',
    'MW_BUY',
    'MW_SELL',
    'DC_MCP_WAVG',
    'DC_AUCTION_TYPE_FIRST',
    'DC_AUCTION_MONTH_FIRST',
    'FW_DC_FIRST',
    'FLOWMONTH',
    # 'AUCTION_ID',
    'AUCTION_MONTH',
    'AUCTION_TYPE',
    'FW',
    'MCP',
    'CONG',
    # 'FUTURE_AUCTION_ID',
    'FUTURE_AUCTION_MONTH',
    'FUTURE_AUCTION_TYPE',
    'FUTURE_FW',
    'FUTURE_MCP'
])

df_sample.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FLOWMONTH,AUCTION_MONTH,AUCTION_TYPE,FW,MCP,CONG,FUTURE_AUCTION_MONTH,FUTURE_AUCTION_TYPE,FUTURE_FW,FUTURE_MCP
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,2023-04-01,2023-02-01,Mar,1,-0.07,0.2016,2023-03-01,Apr,0.0,1.05
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,2023-08-01,2023-05-01,Jun,2,23.79,46.4896,2023-07-01,Aug,0.0,19.41
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,2023-08-01,2023-05-01,Jun,2,23.79,46.4896,2023-06-01,Jul,1.0,20.41
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,2023-08-01,2023-06-01,Jul,1,20.41,46.4896,2023-07-01,Aug,0.0,19.41
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,2023-12-01,2023-08-01,Sep,3,-5.88,-0.4024,2023-11-01,Dec,0.0,-4.56


# Apply bid strategy

In [6]:
df_bids = df_sample.copy()
df_bids['AUCTION_MONTH'] = pd.to_datetime(df_bids['AUCTION_MONTH'], format='%Y-%m-%d')
df_bids['FUTURE_AUCTION_MONTH'] = pd.to_datetime(df_bids['FUTURE_AUCTION_MONTH'], format='%Y-%m-%d')
df_bids['FLOWMONTH'] = pd.to_datetime(df_bids['FLOWMONTH'], format='%Y-%m-%d')

## Want to bid at a "premium" to DC: means bidding higher on long and bidding lower on short
conditions = [
    (df_bids['MW_BUY'] >= df_bids['MW_SELL']) & (df_bids['DC_MCP_WAVG'] > 0), ## DC Buy, PF MCP --> go long
    (df_bids['MW_BUY'] >= df_bids['MW_SELL']) & (df_bids['DC_MCP_WAVG'] < 0), ## DC Buy, CF MCP --> go short
    (df_bids['MW_BUY'] < df_bids['MW_SELL']) & (df_bids['DC_MCP_WAVG'] > 0), ## DC Sell, PF MCP --> go short
    (df_bids['MW_BUY'] < df_bids['MW_SELL']) & (df_bids['DC_MCP_WAVG'] < 0) ## DC Sell, CF MCP --> go long
]

choices = [
    (df_bids['DC_MCP_WAVG'] * 1.2), ## Long: bid 20% higher than DC's clearing price
    (df_bids['DC_MCP_WAVG'] * 0.8), ## Short: bid 20% lower than DC's clearing price
    (df_bids['DC_MCP_WAVG'] * 0.8), ## Short: bid 20% lower than DC's clearing price
    (df_bids['DC_MCP_WAVG'] * 1.2), ## Long: bid 20% higher than DC's clearing price
]

df_bids['BID_ENTRY'] = np.select(conditions, choices, default=np.nan)
df_bids = df_bids[~df_bids['BID_ENTRY'].isna()]

df_bids['ENTRY_TYPE'] = np.where(df_bids['MW_BUY'] >= df_bids['MW_SELL'], 'BUY', 'SELL')
df_bids['FLOW_TYPE'] = np.where(df_bids['MCP'] >= 0, 'PF', 'CF')

## Get an indicator for whether entry cleared
df_bids['CLEARED_ENTRY'] = np.where(
    (df_bids['ENTRY_TYPE'] == 'BUY') & (df_bids['BID_ENTRY'] > df_bids['MCP']) | 
    (df_bids['ENTRY_TYPE'] == 'SELL') & (df_bids['BID_ENTRY'] < df_bids['MCP']),
    1,
    0
).astype(int)

df_bids.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,MCP,CONG,FUTURE_AUCTION_MONTH,FUTURE_AUCTION_TYPE,FUTURE_FW,FUTURE_MCP,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,-0.07,0.2016,2023-03-01,Apr,0.0,1.05,-12.912000,BUY,CF,0
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,23.79,46.4896,2023-07-01,Aug,0.0,19.41,26.358201,BUY,PF,1
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,23.79,46.4896,2023-06-01,Jul,1.0,20.41,26.358201,BUY,PF,1
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,20.41,46.4896,2023-07-01,Aug,0.0,19.41,26.358201,BUY,PF,1
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,-5.88,-0.4024,2023-11-01,Dec,0.0,-4.56,-21.560000,BUY,CF,0


In [7]:
## We can also tabulate exits now. If our bid clear, the OFFER_EXIT is based on MCP (which is entry cost)

# df_potential_trades['OFFER_EXIT'] = df_potential_trades['MCP_ENTRY'] * 1.2
# df_potential_trades['CLEARED_EXIT'] = (df_potential_trades['OFFER_EXIT'] < df_potential_trades['FUTURE_MCP']).astype(int) ## this logic assumes only selling a previous BUY

## Want to try to exit at a greater premium if we are more forward from flow
## If forward >= 2 --> 30% premium, FW1 --> 20%, FW0 --> exit at cost
df_bids['OFFER_PREMIUM'] = np.select(
    [df_bids['FUTURE_FW'] >= 2,
     df_bids['FUTURE_FW'] == 1,
     df_bids['FUTURE_FW'] == 0],
     [30, 20, 0]
) 

## Want to bid at a "premium" to DC: means bidding higher on long and bidding lower on short
conditions_exit = [
    (df_bids['ENTRY_TYPE'] == 'BUY') & (df_bids['FLOW_TYPE'] == 'PF'), ## Long
    (df_bids['ENTRY_TYPE'] == 'BUY') & (df_bids['FLOW_TYPE'] == 'CF'), ## Short
    (df_bids['ENTRY_TYPE'] == 'SELL') & (df_bids['FLOW_TYPE'] == 'PF'), ## Short
    (df_bids['ENTRY_TYPE'] == 'SELL') & (df_bids['FLOW_TYPE'] == 'CF') ## Long
]

choices_exit = [
    (df_bids['MCP'] * (1 + df_bids['OFFER_PREMIUM']/100)), ## Long: sell for more than we bought
    (df_bids['MCP'] * (1 - df_bids['OFFER_PREMIUM']/100)), ## Short: buy back for less than we bought
    (df_bids['MCP'] * (1 - df_bids['OFFER_PREMIUM']/100)), ## Short: buy back for less than we bought (want to buy it back for less than we sold it)
    (df_bids['MCP'] * (1 + df_bids['OFFER_PREMIUM']/100)), ## Long: sell for more than we bought (want someone to pay us more to take it back)
]

## Get an indicator for whether entry cleared
df_bids['OFFER_EXIT'] = np.select(conditions_exit, choices_exit, default=np.nan)
df_bids['EXIT_TYPE'] = np.where(
    df_bids['ENTRY_TYPE'] == 'BUY', 'SELL', 'BUY'
)
## Get an indicator for whether entry cleared
df_bids['CLEARED_EXIT'] = np.where(
    (df_bids['EXIT_TYPE'] == 'BUY') & (df_bids['OFFER_EXIT'] > df_bids['FUTURE_MCP']) | 
    (df_bids['EXIT_TYPE'] == 'SELL') & (df_bids['OFFER_EXIT'] < df_bids['FUTURE_MCP']),
    1,
    0
).astype(int)

df_bids.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_FW,FUTURE_MCP,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_PREMIUM,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,0.0,1.05,-12.912000,BUY,CF,0,0,-0.070,SELL,1
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0.0,19.41,26.358201,BUY,PF,1,0,23.790,SELL,0
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,1.0,20.41,26.358201,BUY,PF,1,20,28.548,SELL,0
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0.0,19.41,26.358201,BUY,PF,1,0,20.410,SELL,0
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,0.0,-4.56,-21.560000,BUY,CF,0,0,-5.880,SELL,1


# Get portfolio outcomes

## Get trades and holdings

In [8]:
## df_bids is really a big set of potential trades

## To look at our portfolio we will do the following iteratively for each AUCTION_MONTH:
## 1) Consider potential trades:
##      a) Consider path-flowmonths that are not in our holdings
##      b) Subset to FW >= 3
## 2) See what clears, add it to *Holdings*
## 3) Get costs and revenue
## 4) Move on to the next AUCTION_MONTH & repeat

In [9]:
df_bids.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_FW,FUTURE_MCP,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_PREMIUM,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,0.0,1.05,-12.912000,BUY,CF,0,0,-0.070,SELL,1
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0.0,19.41,26.358201,BUY,PF,1,0,23.790,SELL,0
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,1.0,20.41,26.358201,BUY,PF,1,20,28.548,SELL,0
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0.0,19.41,26.358201,BUY,PF,1,0,20.410,SELL,0
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,0.0,-4.56,-21.560000,BUY,CF,0,0,-5.880,SELL,1


In [10]:
sorted_auction_months = sorted(df_bids['AUCTION_MONTH'].unique())
df_holdings = pd.DataFrame(columns=df_bids.columns)

for auction_month in sorted_auction_months:

    ## 1) Consider potential trades
    ## 1a) Get broad set of potential trades that we are not already holding
    df_potential_trades = df_bids[
        (df_bids['AUCTION_MONTH'] == auction_month) & 
        (~df_bids['ID_PATH_FLOWMONTH'].isin(df_holdings['ID_PATH_FLOWMONTH']))
    ]

    # df_potential_trades[~df_potential_trades['FUTURE_FW'].isna()] ## don't drop these, just have to hold (unless we remap these nodes)

    ## 1b) Only consider trades with enough opportunities to sell
    df_potential_trades = df_potential_trades[df_potential_trades['FW']>=3]
    if df_potential_trades.empty: continue ## Need this because, for example, standing in March, there are no FWs satisfying FW > 3 (planning year is June-June)
    
    ## 2) See what clears and add it to holdings
    ## Only keep what clears
    df_potential_trades = df_potential_trades[df_potential_trades['CLEARED_ENTRY'] == 1]
    
    ## 3) Get where we exit trades
    ## Sort by CLEARED_EXIT and then FUTURE_AUCTION_MONTH. Gives us the first thing to clear or the first record
    df_sorted = df_potential_trades.sort_values(by=['ID_PATH_FLOWMONTH', 'AUCTION_MONTH', 'CLEARED_EXIT', 'FUTURE_AUCTION_MONTH'], ascending=[True, True, False, True])
    df_first_rows = df_sorted.groupby(['ID_PATH_FLOWMONTH', 'AUCTION_MONTH']).first().reset_index()
    ## Handling cases where everything is empty <-- (this shouldn't happen)
    df_first_rows = df_first_rows.dropna(how='all')
    if df_first_rows.empty: 
        print(f"Month with empty df_first_rows: {auction_month}")
        continue

    ## Add the kept trades to holdings
    df_holdings = pd.concat([df_holdings, df_first_rows]).reset_index(drop=True)



/var/folders/4j/hkr47ph95yg0gwf7gpdpbc600000gn/T/ipykernel_30213/3996778154.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_holdings = pd.concat([df_holdings, df_first_rows]).reset_index(drop=True)


In [11]:
## Get cost and revenue
df_holdings.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_FW,FUTURE_MCP,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_PREMIUM,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT
0,-9222893321979192545,PJMISO,OFF7X8,Obligation,71856761,34887787,DOM:86,DOM:15,DOM,DOM,...,7.0,-26.41,-37.637525,BUY,CF,1,30,-27.930,SELL,1
1,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,METED:39,METED:39,METED,METED,...,1.0,-960.00,-870.494062,SELL,CF,1,20,-748.152,BUY,1
2,-9221875954374263028,PJMISO,WEPEAK,Obligation,51241,1356162213,JCPL:9,PSEG:12,JCPL,PSEG,...,0.0,-15.11,-10.719378,BUY,CF,1,0,-22.940,SELL,1
3,-9221774299757105218,PJMISO,ONPEAK,Obligation,135389799,338269,APS:66,338269,APS,PEPCO,...,9.0,846.35,537.825270,BUY,PF,1,30,584.532,SELL,1
4,-9221237361624466890,PJMISO,ONPEAK,Obligation,50403,2155502045,PSEG:40,PSEG:51,PSEG,PSEG,...,6.0,113.13,177.929816,BUY,PF,1,30,109.824,SELL,1


## Calculate revenue

In [12]:
df_revenue = df_holdings.filter(items=[
    'ID_PATH_FLOWMONTH',
    'ISO',
    'PEAKTYPE',
    'HEDGETYPE',
    'SOURCEID',
    'SINKID',
    'CLUSTERID_LOW_SRC',
    'CLUSTERID_LOW_SNK',
    'SOURCE_ZONE',
    'SINK_ZONE',
    'FLOWMONTH',
    'AUCTION_MONTH',
    'AUCTION_TYPE',
    'FW',
    'MCP',
    'CONG',
    'FUTURE_AUCTION_MONTH',
    'FUTURE_AUCTION_TYPE',
    'FUTURE_FW',
    'FUTURE_MCP',
    'BID_ENTRY',
    'ENTRY_TYPE',
    'FLOW_TYPE',
    'CLEARED_ENTRY',
    'OFFER_EXIT',
    'EXIT_TYPE',
    'CLEARED_EXIT'
]).copy()

df_revenue.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_AUCTION_TYPE,FUTURE_FW,FUTURE_MCP,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT
0,-9222893321979192545,PJMISO,OFF7X8,Obligation,71856761,34887787,DOM:86,DOM:15,DOM,DOM,...,Oct,7.0,-26.41,-37.637525,BUY,CF,1,-27.930,SELL,1
1,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,METED:39,METED:39,METED,METED,...,Nov,1.0,-960.00,-870.494062,SELL,CF,1,-748.152,BUY,1
2,-9221875954374263028,PJMISO,WEPEAK,Obligation,51241,1356162213,JCPL:9,PSEG:12,JCPL,PSEG,...,Apr,0.0,-15.11,-10.719378,BUY,CF,1,-22.940,SELL,1
3,-9221774299757105218,PJMISO,ONPEAK,Obligation,135389799,338269,APS:66,338269,APS,PEPCO,...,Aug,9.0,846.35,537.825270,BUY,PF,1,584.532,SELL,1
4,-9221237361624466890,PJMISO,ONPEAK,Obligation,50403,2155502045,PSEG:40,PSEG:51,PSEG,PSEG,...,Nov,6.0,113.13,177.929816,BUY,PF,1,109.824,SELL,1


In [13]:
## Need to assert this
df_revenue[df_revenue['CLEARED_ENTRY']==0].head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_AUCTION_TYPE,FUTURE_FW,FUTURE_MCP,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT


In [14]:
df_revenue['COST'] = df_revenue['MCP']

df_revenue['REVENUE_MONTH'] = np.where(
    df_revenue['CLEARED_EXIT']==1,
    df_revenue['FUTURE_AUCTION_MONTH'],
    df_revenue['FLOWMONTH']
)

df_revenue['REVENUE'] = np.where(
    df_revenue['CLEARED_EXIT']==1,
    df_revenue['FUTURE_MCP'],
    df_revenue['CONG']
)

df_revenue['PROFIT'] = np.where(
    df_revenue['ENTRY_TYPE'] == 'BUY',
    df_revenue['REVENUE'] - df_revenue['COST'],
    df_revenue['COST'] - df_revenue['REVENUE']
)

In [15]:
df_revenue.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT,COST,REVENUE_MONTH,REVENUE,PROFIT
0,-9222893321979192545,PJMISO,OFF7X8,Obligation,71856761,34887787,DOM:86,DOM:15,DOM,DOM,...,BUY,CF,1,-27.930,SELL,1,-39.90,2023-09-01,-26.41,13.49
1,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,METED:39,METED:39,METED,METED,...,SELL,CF,1,-748.152,BUY,1,-623.46,2023-10-01,-960.00,336.54
2,-9221875954374263028,PJMISO,WEPEAK,Obligation,51241,1356162213,JCPL:9,PSEG:12,JCPL,PSEG,...,BUY,CF,1,-22.940,SELL,1,-22.94,2024-03-01,-15.11,7.83
3,-9221774299757105218,PJMISO,ONPEAK,Obligation,135389799,338269,APS:66,338269,APS,PEPCO,...,BUY,PF,1,584.532,SELL,1,449.64,2023-07-01,846.35,396.71
4,-9221237361624466890,PJMISO,ONPEAK,Obligation,50403,2155502045,PSEG:40,PSEG:51,PSEG,PSEG,...,BUY,PF,1,109.824,SELL,1,84.48,2023-10-01,113.13,28.65


# Get some portfolios to look at

In [16]:
df_holdings.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_FW,FUTURE_MCP,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_PREMIUM,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT
0,-9222893321979192545,PJMISO,OFF7X8,Obligation,71856761,34887787,DOM:86,DOM:15,DOM,DOM,...,7.0,-26.41,-37.637525,BUY,CF,1,30,-27.930,SELL,1
1,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,METED:39,METED:39,METED,METED,...,1.0,-960.00,-870.494062,SELL,CF,1,20,-748.152,BUY,1
2,-9221875954374263028,PJMISO,WEPEAK,Obligation,51241,1356162213,JCPL:9,PSEG:12,JCPL,PSEG,...,0.0,-15.11,-10.719378,BUY,CF,1,0,-22.940,SELL,1
3,-9221774299757105218,PJMISO,ONPEAK,Obligation,135389799,338269,APS:66,338269,APS,PEPCO,...,9.0,846.35,537.825270,BUY,PF,1,30,584.532,SELL,1
4,-9221237361624466890,PJMISO,ONPEAK,Obligation,50403,2155502045,PSEG:40,PSEG:51,PSEG,PSEG,...,6.0,113.13,177.929816,BUY,PF,1,30,109.824,SELL,1


In [17]:
flowmonths = sorted(df_holdings['FLOWMONTH'].unique())
print(flowmonths)

[Timestamp('2023-09-01 00:00:00'), Timestamp('2023-10-01 00:00:00'), Timestamp('2023-11-01 00:00:00'), Timestamp('2023-12-01 00:00:00'), Timestamp('2024-01-01 00:00:00'), Timestamp('2024-02-01 00:00:00'), Timestamp('2024-03-01 00:00:00'), Timestamp('2024-04-01 00:00:00'), Timestamp('2024-05-01 00:00:00')]


In [34]:
thismonth = flowmonths[1]
print(thismonth)

df_takealook = df_holdings[df_holdings['FLOWMONTH']==flowmonths[1]].copy()
df_takealook['PROFIT'] = np.where(
    df_takealook['ENTRY_TYPE']=='BUY',
    (df_takealook['CONG']-df_takealook['MCP']) * df_takealook['CLEARED_ENTRY'],
    (df_takealook['MCP']-df_takealook['CONG'])  * df_takealook['CLEARED_ENTRY']
)
print(df_takealook['PROFIT'].sum())

df_takealook = df_takealook.filter(items=[
    'FLOWMONTH',
    'AUCTION_MONTH',
    'PEAKTYPE',
    'HEDGETYPE',
    'SOURCEID',
    'SINKID',
    'ENTRY_TYPE',
    'BID_ENTRY',
    # 'MWS_ROUNDED2'
])

print(df_takealook.dtypes)

## YE auction_dates are one month ahead of VB auction_months
df_takealook['AUCTION_MONTH'] = df_takealook['AUCTION_MONTH'] + pd.DateOffset(months=1)
df_takealook.rename(columns={'AUCTION_MONTH': 'AUCTION_DATE'}, inplace=True)

df_takealook.rename(columns={'FLOWMONTH': 'CONTRACT_STARTDATE'}, inplace=True)
df_takealook.rename(columns={'ENTRY_TYPE': 'TRADE_TYPE'}, inplace=True)
df_takealook.rename(columns={'BID_ENTRY': 'BID'}, inplace=True)
# df_takealook.rename(columns={'MWS_ROUNDED2': 'PATHSIZE'}, inplace=True)
df_takealook['CONTRACT_TYPE'] = "M"

# df_takealook.to_csv('~/Downloads/sasha_bidstrat_Aug24.csv', index=False)

df_takealook.head()

2023-10-01 00:00:00
-53669.2396
FLOWMONTH        datetime64[ns]
AUCTION_MONTH    datetime64[ns]
PEAKTYPE                 object
HEDGETYPE                object
SOURCEID                 object
SINKID                   object
ENTRY_TYPE               object
BID_ENTRY               float64
dtype: object


,CONTRACT_STARTDATE,AUCTION_DATE,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,TRADE_TYPE,BID,CONTRACT_TYPE
8,2023-10-01,2023-06-01,OFF7X8,Obligation,50641,1048039,SELL,-43.531898,M
17,2023-10-01,2023-06-01,ONPEAK,Obligation,33092311,1268571933,BUY,163.796453,M
23,2023-10-01,2023-06-01,WEPEAK,Obligation,1348263397,2156109754,BUY,-12.038309,M
26,2023-10-01,2023-06-01,ONPEAK,Obligation,1123180723,32412317,SELL,20.339806,M
42,2023-10-01,2023-06-01,OFF7X8,Obligation,34497127,123901555,BUY,114.756304,M


# Profit and Loss

## PnL by flow month

In [41]:
df_pnl_flowmonth = df_revenue.groupby('FLOWMONTH').agg(
    PROFIT = ('PROFIT', 'sum'),
    TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count')
).reset_index()

df_pnl_flowmonth = df_pnl_flowmonth.sort_values(by=['FLOWMONTH'])
df_pnl_flowmonth['PROFIT_CUMSUM'] = df_pnl_flowmonth['PROFIT'].cumsum()

df_pnl_flowmonth['PROFIT_SCALED'] = df_pnl_flowmonth['PROFIT'] / df_pnl_flowmonth['TRADE_COUNT']
df_pnl_flowmonth['PROFITSCALED_CUMSUM'] = df_pnl_flowmonth['PROFIT_SCALED'].cumsum()

df_pnl_flowmonth.head(12)

,FLOWMONTH,PROFIT,TRADE_COUNT,PROFIT_CUMSUM,PROFIT_SCALED,PROFITSCALED_CUMSUM
0,2023-09-01,-33429.1412,2507,-33429.1412,-13.334320,-13.334320
1,2023-10-01,-29562.4072,3186,-62991.5484,-9.278847,-22.613168
2,2023-11-01,3272.9162,4070,-59718.6322,0.804156,-21.809011
3,2023-12-01,41890.3480,4452,-17828.2842,9.409332,-12.399679
4,2024-01-01,172462.7448,4520,154634.4606,38.155475,25.755796
5,2024-02-01,201279.6820,5099,355914.1426,39.474344,65.230140
6,2024-03-01,71780.8148,6532,427694.9574,10.989102,76.219242
7,2024-04-01,128455.8084,7506,556150.7658,17.113750,93.332992
8,2024-05-01,278296.8000,7911,834447.5658,35.178460,128.511453


In [42]:
df_pnl_flowmonth = df_revenue.groupby(['FLOWMONTH']).agg(
    PROFIT_TOTAL = ('PROFIT', 'sum'),
    TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count'),
    PROFIT_PF = ('PROFIT', lambda x: x[df_revenue['FLOW_TYPE'] == 'PF'].sum()),
    TRADE_COUNT_PF = ('ID_PATH_FLOWMONTH', lambda x: x[df_revenue['FLOW_TYPE'] == 'PF'].count()),
    PROFIT_CF = ('PROFIT', lambda x: x[df_revenue['FLOW_TYPE'] == 'CF'].sum()),
    TRADE_COUNT_CF = ('ID_PATH_FLOWMONTH', lambda x: x[df_revenue['FLOW_TYPE'] == 'CF'].count())
).reset_index()

df_pnl_flowmonth = df_pnl_flowmonth.sort_values(by=['FLOWMONTH'])
df_pnl_flowmonth['PROFIT_TOTAL_CUMSUM'] = df_pnl_flowmonth['PROFIT_TOTAL'].cumsum()
df_pnl_flowmonth['PROFIT_PF_CUMSUM'] = df_pnl_flowmonth['PROFIT_PF'].cumsum()
df_pnl_flowmonth['PROFIT_CF_CUMSUM'] = df_pnl_flowmonth['PROFIT_CF'].cumsum()

## Return per MW
df_pnl_flowmonth['PROFIT_PER_MW'] = df_pnl_flowmonth['PROFIT_TOTAL'] / df_pnl_flowmonth['TRADE_COUNT']
df_pnl_flowmonth['PF_PROFIT_PER_MW'] = df_pnl_flowmonth['PROFIT_PF'] / df_pnl_flowmonth['TRADE_COUNT_PF']
df_pnl_flowmonth['CF_PROFIT_PER_MW'] = df_pnl_flowmonth['PROFIT_CF'] / df_pnl_flowmonth['TRADE_COUNT_CF']


## Try to format?
df_pnl_flowmonth['PROFIT_TOTAL'] = df_pnl_flowmonth['PROFIT_TOTAL'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_TOTAL_CUMSUM'] = df_pnl_flowmonth['PROFIT_TOTAL_CUMSUM'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_PF'] = df_pnl_flowmonth['PROFIT_PF'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_PF_CUMSUM'] = df_pnl_flowmonth['PROFIT_PF_CUMSUM'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_CF'] = df_pnl_flowmonth['PROFIT_CF'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_CF_CUMSUM'] = df_pnl_flowmonth['PROFIT_CF_CUMSUM'].apply(lambda x: "${:,.2f}".format(x))

df_pnl_flowmonth['PROFIT_PER_MW'] = df_pnl_flowmonth['PROFIT_PER_MW'].apply(lambda x: "$/MW {:,.2f}".format(x))
df_pnl_flowmonth['PF_PROFIT_PER_MW'] = df_pnl_flowmonth['PF_PROFIT_PER_MW'].apply(lambda x: "$/MW {:,.2f}".format(x))
df_pnl_flowmonth['CF_PROFIT_PER_MW'] = df_pnl_flowmonth['CF_PROFIT_PER_MW'].apply(lambda x: "$/MW {:,.2f}".format(x))

df_pnl_flowmonth = df_pnl_flowmonth[
    ['FLOWMONTH', 
     'PROFIT_TOTAL', 'PROFIT_TOTAL_CUMSUM',
     'PROFIT_PER_MW',
     'PROFIT_PF', 'PROFIT_PF_CUMSUM', 'PF_PROFIT_PER_MW',
     'PROFIT_CF', 'PROFIT_CF_CUMSUM', 'CF_PROFIT_PER_MW',
     'TRADE_COUNT', 
     'TRADE_COUNT_PF', 'TRADE_COUNT_CF']
]

df_pnl_flowmonth.head(12)

,FLOWMONTH,PROFIT_TOTAL,PROFIT_TOTAL_CUMSUM,PROFIT_PER_MW,PROFIT_PF,PROFIT_PF_CUMSUM,PF_PROFIT_PER_MW,PROFIT_CF,PROFIT_CF_CUMSUM,CF_PROFIT_PER_MW,TRADE_COUNT,TRADE_COUNT_PF,TRADE_COUNT_CF
0,2023-09-01,"$-33,429.14","$-33,429.14",$/MW -13.33,"$-46,607.75","$-46,607.75",$/MW -54.38,"$13,178.61","$13,178.61",$/MW 7.99,2507,857,1650
1,2023-10-01,"$-29,562.41","$-62,991.55",$/MW -9.28,"$-34,054.95","$-80,662.70",$/MW -27.91,"$4,492.54","$17,671.15",$/MW 2.29,3186,1220,1966
2,2023-11-01,"$3,272.92","$-59,718.63",$/MW 0.80,"$9,880.80","$-70,781.89",$/MW 6.18,"$-6,607.89","$11,063.26",$/MW -2.67,4070,1598,2472
3,2023-12-01,"$41,890.35","$-17,828.28",$/MW 9.41,"$-71,481.57","$-142,263.46",$/MW -44.18,"$113,371.91","$124,435.18",$/MW 40.00,4452,1618,2834
4,2024-01-01,"$172,462.74","$154,634.46",$/MW 38.16,"$48,279.21","$-93,984.24",$/MW 30.21,"$124,183.53","$248,618.71",$/MW 42.50,4520,1598,2922
5,2024-02-01,"$201,279.68","$355,914.14",$/MW 39.47,"$64,780.10","$-29,204.14",$/MW 36.19,"$136,499.58","$385,118.29",$/MW 41.25,5099,1790,3309
6,2024-03-01,"$71,780.81","$427,694.96",$/MW 10.99,"$14,617.80","$-14,586.34",$/MW 5.86,"$57,163.01","$442,281.30",$/MW 14.16,6532,2494,4038
7,2024-04-01,"$128,455.81","$556,150.77",$/MW 17.11,"$19,068.21","$4,481.87",$/MW 6.96,"$109,387.60","$551,668.90",$/MW 22.96,7506,2741,4765
8,2024-05-01,"$278,296.80","$834,447.57",$/MW 35.18,"$145,826.48","$150,308.35",$/MW 48.74,"$132,470.32","$684,139.21",$/MW 26.93,7911,2992,4919


## PnL by zone

In [43]:
## sometimes zone is missing and that throws off the calculation
print(df_revenue['SOURCE_ZONE'].unique())
print(df_revenue['SINK_ZONE'].unique())

['DOM' 'METED' 'JCPL' 'APS' 'PSEG' 'AEP' 'COMED' 'PPL' 'PENELEC' 'ATSI'
 None 'PECO' 'DEOK' 'EKPC' 'AECO' 'DPL' 'OVEC' 'BGE' 'DAY' 'PEPCO' 'DUQ'
 'RECO']
['DOM' 'METED' 'PSEG' 'PEPCO' 'AEP' 'COMED' 'APS' 'PPL' 'PECO' 'PENELEC'
 'ATSI' 'AECO' 'DAY' 'DPL' 'OVEC' None 'DEOK' 'BGE' 'EKPC' 'DUQ' 'JCPL'
 'RECO']


In [44]:
# df_pnl_zone = df_revenue.copy().groupby('SINK_ZONE').agg(
#     PROFIT = ('PROFIT', 'sum'),
#     TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count')
# ).reset_index()

df_pnl_zone = df_revenue.copy().groupby('SOURCE_ZONE').agg(
    PROFIT = ('PROFIT', 'sum'),
    TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count')
).reset_index()

print(f"Are there duplicate IDs?: {df_revenue.duplicated(subset=['ID_PATH_FLOWMONTH']).sum()}")
print(f"Are there missing profits?: {df_revenue['PROFIT'].isnull().sum()}")

df_pnl_zone['PROFIT_TOTAL'] = df_pnl_zone['PROFIT'].sum()

df_pnl_zone['PROFIT_SCALED'] = df_pnl_zone['PROFIT'] / df_pnl_zone['TRADE_COUNT']

df_pnl_zone.head(50)

Are there duplicate IDs?: 0
Are there missing profits?: 0


,SOURCE_ZONE,PROFIT,TRADE_COUNT,PROFIT_TOTAL,PROFIT_SCALED
0,AECO,13396.8739,1055,799355.2108,12.698459
1,AEP,106671.6885,4984,799355.2108,21.402827
2,APS,191291.4226,2895,799355.2108,66.076484
3,ATSI,31292.7922,2061,799355.2108,15.183305
4,BGE,47524.9840,612,799355.2108,77.655203
5,COMED,-29718.7229,7515,799355.2108,-3.954587
6,DAY,11516.6130,542,799355.2108,21.248363
7,DEOK,-8782.0272,1484,799355.2108,-5.917808
8,DOM,106417.8728,4610,799355.2108,23.084137
9,DPL,29840.5285,2135,799355.2108,13.976828


## PnL in a cash flow sense

In [48]:
df_revenue.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT,COST,REVENUE_MONTH,REVENUE,PROFIT
0,-9222893321979192545,PJMISO,OFF7X8,Obligation,71856761,34887787,DOM:86,DOM:15,DOM,DOM,...,BUY,CF,1,-27.930,SELL,1,-39.90,2023-09-01,-26.41,13.49
1,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,METED:39,METED:39,METED,METED,...,SELL,CF,1,-748.152,BUY,1,-623.46,2023-10-01,-960.00,336.54
2,-9221875954374263028,PJMISO,WEPEAK,Obligation,51241,1356162213,JCPL:9,PSEG:12,JCPL,PSEG,...,BUY,CF,1,-22.940,SELL,1,-22.94,2024-03-01,-15.11,7.83
3,-9221774299757105218,PJMISO,ONPEAK,Obligation,135389799,338269,APS:66,338269,APS,PEPCO,...,BUY,PF,1,584.532,SELL,1,449.64,2023-07-01,846.35,396.71
4,-9221237361624466890,PJMISO,ONPEAK,Obligation,50403,2155502045,PSEG:40,PSEG:51,PSEG,PSEG,...,BUY,PF,1,109.824,SELL,1,84.48,2023-10-01,113.13,28.65


In [49]:
df_entry = df_revenue.filter(items=[
    'ID_PATH_FLOWMONTH',
    'ISO',
    'PEAKTYPE',
    'HEDGETYPE',
    'SOURCEID',
    'SINKID',
    'FLOWMONTH',
    'ENTRY_TYPE',
    'AUCTION_MONTH',
    'COST'
]).copy()
df_entry.rename(columns={'AUCTION_MONTH': 'MONTH_CF'}, inplace=True)
df_entry.rename(columns={'COST': 'CASHFLOW'}, inplace=True)
## If it was a long FW position, cost is negative (out-flow)
df_entry['CASHFLOW'] = np.where(
    df_entry['ENTRY_TYPE'] == 'SELL',
    df_entry['CASHFLOW'],
    df_entry['CASHFLOW']*(-1)
)

df_exit = df_revenue.filter(items=[
    'ID_PATH_FLOWMONTH',
    'ISO',
    'PEAKTYPE',
    'HEDGETYPE',
    'SOURCEID',
    'SINKID',
    'FLOWMONTH',
    'ENTRY_TYPE',
    'REVENUE_MONTH',
    'REVENUE'
]).copy()
df_exit.rename(columns={'REVENUE_MONTH': 'MONTH_CF'}, inplace=True)
df_exit.rename(columns={'REVENUE': 'CASHFLOW'}, inplace=True)

## If it was a long FW position, revenue is positive (in-flow)
df_exit['CASHFLOW'] = np.where(
    df_exit['ENTRY_TYPE'] == 'BUY',
    df_exit['CASHFLOW'],
    df_exit['CASHFLOW']*(-1)
)

df_pnl_cashflow = pd.concat([df_entry, df_exit]).reset_index(drop=True)
df_pnl_cashflow.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,FLOWMONTH,ENTRY_TYPE,MONTH_CF,CASHFLOW
0,-9222893321979192545,PJMISO,OFF7X8,Obligation,71856761,34887787,2024-05-01,BUY,2023-05-01,39.90
1,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,2023-12-01,SELL,2023-05-01,-623.46
2,-9221875954374263028,PJMISO,WEPEAK,Obligation,51241,1356162213,2024-04-01,BUY,2023-05-01,22.94
3,-9221774299757105218,PJMISO,ONPEAK,Obligation,135389799,338269,2024-05-01,BUY,2023-05-01,-449.64
4,-9221237361624466890,PJMISO,ONPEAK,Obligation,50403,2155502045,2024-05-01,BUY,2023-05-01,-84.48


In [50]:
df_pnl_cashflow = df_pnl_cashflow.groupby('MONTH_CF').agg(
    PROFIT = ('CASHFLOW', 'sum'),
    TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count')
).reset_index()

df_pnl_cashflow = df_pnl_cashflow.sort_values(by=['MONTH_CF'])
df_pnl_cashflow['PROFIT_CUMSUM'] = df_pnl_cashflow['PROFIT'].cumsum()

df_pnl_cashflow['PROFIT_SCALED'] = df_pnl_cashflow['PROFIT'] / df_pnl_cashflow['TRADE_COUNT']
df_pnl_cashflow['PROFITSCALED_CUMSUM'] = df_pnl_cashflow['PROFIT_SCALED'].cumsum()

df_pnl_cashflow.head(20)

,MONTH_CF,PROFIT,TRADE_COUNT,PROFIT_CUMSUM,PROFIT_SCALED,PROFITSCALED_CUMSUM
0,2023-05-01,-482997.6100,21995,-482997.6100,-21.959428,-21.959428
1,2023-06-01,-145194.5600,13533,-628192.1700,-10.728926,-32.688354
2,2023-07-01,-4960.9800,11723,-633153.1500,-0.423183,-33.111537
3,2023-08-01,199458.1300,8160,-433695.0200,24.443398,-8.668139
4,2023-09-01,170748.8888,7315,-262946.1312,23.342295,14.674156
5,2023-10-01,295144.8228,6924,32198.6916,42.626346,57.300502
6,2023-11-01,-61018.8838,5850,-28820.1922,-10.430578,46.869924
7,2023-12-01,116006.0280,4982,87185.8358,23.285032,70.154956
8,2024-01-01,-6406.3852,3239,80779.4506,-1.977890,68.177066
9,2024-02-01,-63093.7380,2165,17685.7126,-29.142604,39.034462


In [25]:
print(f"Sum of profit column from flow-month PnL: {df_pnl_flowmonth['PROFIT'].sum()}")
print(f"Sum of profit column from cash-flow-month PnL: {df_pnl_cashflow['PROFIT'].sum()}")

Sum of profit column from flow-month PnL: 834447.5658
Sum of profit column from cash-flow-month PnL: 834447.5658000002
